In [ ]:
%matplotlib widget
import warnings
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
class CartesianGrid:
    def __init__(self, lx, ly, nx, ny):
        print("Initializing grid.")
        if ((nx%2)+(ny%2)) > 0:
            raise Exception("Number of points along each coordinate should be multiple of 2 for successful padding.")

        self.lx = lx
        self.ly = ly
        self.nx = nx
        self.ny = ny

        self.lxp = (lx * 3)//2
        self.lyp = (ly * 3)//2
        self.nxp = (nx * 3)//2
        self.nyp = (ny * 3)//2

        self.x  = np.linspace(0, lx, nx)
        self.y  = np.linspace(0, ly, ny)
        self.dx = self.x[1]-self.x[0]
        self.dy = self.y[1]-self.y[0]
        self.X, self.Y = np.meshgrid(self.x, self.y)


In [ ]:
def TaylorVortex(grid, x0, y0, a0, Umax):
    # | -n,n   |-(n-1),n  |...|0,n  |...|n,n  |
    # | -n,n-1 |-(n-1),n-1|...|0,n-1|...|n,n-1|
    #      :       :         :  :          :
    # | -n,0   |-(n-1),0  |...|0,0  |...|n,0  |
    #    :         :              :        :
    # | -n,-n  |-(n-1),-n |...|0,-n |...|n,-n |

    # Due to periodic domain -n to n domain effects 
    # along x and y directions are considered while 
    # computing omega in (0,0) i.e. current domain.
    n = 1   
    omega = 0
    for i in range(-n,n+1):
        for j in range(-n,n+1):
            h = x0 + i* grid.lx
            k = y0 + j* grid.ly
            r2 = (grid.X - h)**2 + (grid.Y-k)**2
            omega = omega + Umax/a0*(2-r2/a0**2)*np.exp(0.5*(1-r2/a0**2))
    omegahat = np.fft.fft2(omega)
    
    return omegahat

In [ ]:
def rk4(f , x0, t, dt):
    k1 = f(t, x0)
    k2 = f(t+0.5*dt, x0 + 0.5*dt*k1)
    k3 = f(t+0.5*dt, x0 + 0.5*dt*k2)
    k4 = f(t+dt    , x0 +     dt*k3)
    xn = x0 + dt/6*(k1+2*(k2+k3)+k4)
    return xn

# Here we will solve the vorticity equation

Momentum equation can be written as,

$$
\rho\frac{\partial\vec{u}}{\partial t} 
+ \rho (\vec{u}.\nabla) \vec{u}= 
- \nabla p
+ \vec{g}
+ \mu \nabla^2 \vec{u}
$$

Taking curl of this equation,

$$
\nabla\times\bigg[
\rho\frac{\partial\vec{u}}{\partial t} 
+ \rho (\vec{u}.\nabla) \vec{u}
\bigg]
=
\nabla\times\bigg[
- \nabla p
+ \vec{g}
+ \mu \nabla^2 \vec{u}
\bigg]
$$

Assuming incompressible fluid i.e. $\rho$ is constant and $\nabla . \vec{u} = 0$ and with identity $\nabla \times \vec{u}=\vec{\omega}$

$$
\rho\frac{\partial\vec{\omega}}{\partial t} 
+ \rho \nabla\times [(\vec{u}.\nabla) \vec{u}]
=
- \nabla\times (\nabla p)
+ \nabla\times (\vec{g})
+ \mu \nabla\times (\nabla^2 \vec{u})
$$

using identity $\nabla \times (\nabla \phi) = 0 $ where $\phi$ is scaler function.

$$
\frac{\partial\vec{\omega}}{\partial t} 
+ \nabla\times [\nabla (\vec{u}.\vec{u}) + (\nabla \times \vec{u})\times \vec{u}]
=
\nu \nabla^2 \vec{\omega}
$$

here, $(\vec{u}.\vec{u})$ is a scalar field thus,

$$
\frac{\partial\vec{\omega}}{\partial t} 
+ \nabla\times [\vec{\omega}\times \vec{u}]
=
\nu \nabla^2 \vec{\omega}
$$


we will use,

$$
\nabla\times (\vec{\omega}\times \vec{u}) 
= 
(\vec{u}.\nabla)\vec{\omega}
-(\vec{\omega}.\nabla)\vec{u}
$$

$$
\frac{\partial\vec{\omega}}{\partial t} 
=
\nu \nabla^2 \vec{\omega}
+(\vec{\omega}.\nabla)\vec{u}
- (\vec{u}.\nabla)\vec{\omega}
$$

For 2D case, 
$$\vec{\omega}=\omega(x,y)\hat{k}$$
$$\vec{u}= u \hat{i}+ v \hat{j}$$ 

$$
(\vec{\omega}.\nabla)\vec{u} = 
\omega\frac{\partial}{\partial z} \vec{u} 
= 0 
$$
as $\vec{u}$ is independent of $z$.

Now the equation will contain only $z$ component from all terma as,

$$
\frac{\partial{\omega}}{\partial t} 
=
\nu \bigg(  \frac{\partial^2 \omega}{\partial x^2} +
            \frac{\partial^2 \omega}{\partial y^2}
    \bigg) 
-  \bigg(   u\frac{\partial \omega}{\partial x} +
            v\frac{\partial \omega}{\partial y}
   \bigg)
$$

$$
\boxed{
\frac{\partial{\omega}}{\partial t} 
=
\nu \bigg(  \frac{\partial^2 \omega}{\partial x^2} +
            \frac{\partial^2 \omega}{\partial y^2}
    \bigg) 
-  \bigg(   u\frac{\partial \omega}{\partial x} +
            v\frac{\partial \omega}{\partial y}
   \bigg)
}
\tag{eq:vorticity-physical}
$$

Let, 
$$
\vec{u} = \nabla \times \vec{\psi}(x,y) = 
\begin{bmatrix}
\hat{i}  &  \hat{i} & \hat{i} \\
\frac{\partial}{\partial x} & \frac{\partial}{\partial y} & \frac{\partial}{\partial z} \\
0 & 0 & \psi
\end{bmatrix}
$$

$$
\vec{u} =
\frac{\partial \psi}{\partial y} \hat{i}
- \frac{\partial \psi}{\partial x} \hat{j}
$$

$$
\boxed{
u = \frac{\partial \psi}{\partial y} ,\hspace{0.5cm}
v = - \frac{\partial \psi}{\partial x}
}
\tag{eq:velocity-physical}
$$



$$
\vec{\omega}  = \nabla \times ( \nabla \times \vec{\psi} )
$$

$$
\vec{\omega}  = \nabla ( \nabla . \vec{\psi} )
- \nabla^2 \vec{\psi}
$$

Here,

$$
\nabla.\vec{\psi} = (\hat{i}\frac{\partial}{\partial x}+
                      \hat{j}\frac{\partial}{\partial y} +
                      \hat{k}\frac{\partial}{\partial z}).(\psi \hat{k})
                 = 0
$$
as $\psi$ is function of $x$ and $y$.

$$
\vec{\omega}  = - \nabla^2 \vec{\psi}
$$

as $\vec{\psi} = \psi(x,y)\hat{k}$,

$$
\vec{\omega} = - \bigg( \frac{\partial^2}{\partial x^2}
                       +\frac{\partial^2}{\partial y^2}
                       +\frac{\partial^2}{\partial z^2}
                 \bigg) (\psi \hat{k})
$$



Thus,

$$
\vec{\omega} = - \bigg( \frac{\partial^2\psi}{\partial x^2}
                       +\frac{\partial^2\psi}{\partial y^2}
                 \bigg) \hat{k}
$$

Thus $\vec{\omega}=\omega \hat{k}$ has only $z$ component.
$$
\omega = - \bigg( \frac{\partial^2\psi}{\partial x^2}
                       +\frac{\partial^2\psi}{\partial y^2}
                 \bigg)
$$

For 1D case, vorticity at $x_m$ is,
$$
\omega_m = \sum_{n=0}^{M-1} \hat{\omega}_n e^{2\pi \iota (m n)/M}
$$

here $L$ length domain is divided into $M$ parts.
$$
\omega_m = \sum_{n=0}^{M-1} \hat{\omega}_n e^{\iota (2\pi  n/L) (m L/M)}
$$

$$
\omega_m = \sum_{n=0}^{M-1} \hat{\omega}_n e^{\iota k_n x_m}
$$


Generalizing to 2D case,
$$
\omega(x_p,y_q)=\omega_{p,q} = \sum_{m=0}^{M-1} \sum_{n=0}^{M-1} \hat{\omega}_{m,n} e^{\iota [k_x(m) x_p + k_y(n) y_q]}
$$

$$
\psi_{p,q} = \sum_{m=0}^{M-1} \sum_{n=0}^{M-1} \hat{\psi}_{m,n} e^{\iota [k_x(m) x_p + k_y(n) y_q]}
$$


$$
\bigg(\frac{\partial^2 \psi}{\partial x^2} + \frac{\partial^2 \psi}{\partial y^2} \bigg)_{p,q}= 
-\sum_{m=0}^{M-1} \sum_{n=0}^{M-1} [k_x^2(m) + k_y^2(n)] \hat{\psi}_{m,n} e^{\iota [k_x(m) x_p + k_y(n) y_q]}
$$




$$
\omega_{p,q} = -\bigg(\frac{\partial^2 \psi}{\partial x^2} + \frac{\partial^2 \psi}{\partial y^2} \bigg)= 
\sum_{m=0}^{M-1} \sum_{n=0}^{M-1} [k_x^2(m) + k_y^2(n)] \hat{\psi}_{m,n} e^{\iota [k_x(m) x_p + k_y(n) y_q]}
$$

$$
\hat{\omega}_{m,n} = 
[k_x^2(m) + k_y^2(n)] \hat{\psi}_{m,n} 
$$

$$
\hat{\psi}_{m,n}  = \frac{1}{[k_x^2(m) + k_y^2(n)] }\hat{\omega}_{m,n}
$$


$$
u_{p,q}  = \frac{\partial \psi}{\partial y}= \sum_{m=0}^{M-1} \sum_{n=0}^{M-1} \iota k_y(n) \hat{\psi}_{m,n} e^{\iota [k_x(m) x_p + k_y(n) y_q]}
$$

$$
\boxed{
\hat{u}_{m,n}  = \iota k_y(n) \hat{\psi}_{m,n} 
}
$$



$$
v_{p,q}  =-\frac{\partial \psi}{\partial x}= -\sum_{m=0}^{M-1} \sum_{n=0}^{M-1} \iota k_x(m) \hat{\psi}_{m,n} e^{\iota [k_x(m) x_p + k_y(n) y_q]}
$$

$$
\boxed{
\hat{v}_{m,n}  = - \iota k_x(m) \hat{\psi}_{m,n} 
}
$$

# Collecting all terms that are required to evolve vortivity field

$$
\boxed{
\frac{\partial{\omega}}{\partial t} 
=
\nu \bigg(  \frac{\partial^2 \omega}{\partial x^2} +
            \frac{\partial^2 \omega}{\partial y^2}
    \bigg) 
-  \bigg(   u\frac{\partial \omega}{\partial x} +
            v\frac{\partial \omega}{\partial y}
   \bigg)
}
\tag{eq:vorticity-physical}
$$

$$
\boxed{
\frac{\partial \hat{\omega}_{m,n}}{\partial t}
=
-\nu [ k^2_x(m) +  k^2_y(n)]\hat{\omega}_{m,n}
+ \hat{A}
}
\tag{eq:vorticity-transformed}
$$

Here, Fourier transformed $\hat{A}$ is the advection term.

$$
A = 
-  \bigg(   u\frac{\partial \omega}{\partial x} +
            v\frac{\partial \omega}{\partial y}
   \bigg)
$$

This term has to be carefully compted to avoid aliasing

$$
\frac{\partial \omega}{\partial x} = \omega_x
$$

$$
\hat{\omega}_x(k_x(m), k_y(n)) = \big(\hat{\omega}_x\big)_{m,n} = \iota k_x(m) \hat{\omega}
$$

For 1D case using $2/3$ rule we will pad it, $L>3M/2$ where $M$ is original length of data and $L$ is the legth of data after padding, 

$$
\hat{u} = \{ \hat{u}_{-\frac{M}{2}+1}, \cdots, \hat{u}_{-1}, \hat{u}_{0},  \hat{u}_{1}, \cdots, \hat{u}_{\frac{M}{2}}\}
$$

$$
\hat{u}_{pad} = \{\underbrace{0}_{(\frac{-3M}{4}+1)^{th}}, \cdots, 0, \hat{u}_{-\frac{M}{2}+1}, \cdots, \hat{u}_{-1}, \hat{u}_{0},  \hat{u}_{1}, \cdots, \hat{u}_{\frac{M}{2}}, 0, \cdots, \underbrace{0}_{(\frac{3M}{4})^{th}} \}
$$


when actual Fourier transform is computed the output vector is,
$$
\hat{u} = \{ \hat{u}_{0},  \hat{u}_{1}, \cdots, \hat{u}_{\frac{M}{2}}, \hat{u}_{-\frac{M}{2}+1}, \cdots, \hat{u}_{-1}\}
$$

which is equivalent to,
$$
\hat{u} = \{ \hat{u}_{0},  \hat{u}_{1}, \cdots, \hat{u}_{\frac{M}{2}}, \hat{u}_{\frac{M}{2}+1}, \cdots, \hat{u}_{M-1}\}
$$

as $\hat{u}_m = \hat{u}_{m\pm M}$

Thus,
$$
\hat{u}_{pad} = \{\underbrace{0, \cdots, 0, \hat{u}_{-\frac{M}{2}+1}, \cdots, \hat{u}_{-1}}_{\text{adding $M$ to index}}, \hat{u}_{0},  \hat{u}_{1}, \cdots, \hat{u}_{\frac{M}{2}}, 0, \cdots, \underbrace{0}_{(\frac{3M}{4})^{th}} \}
$$

$$
\hat{u}_{pad} = \{\underbrace{0, \cdots, 0, \hat{u}_{-\frac{M}{2}+1}, \cdots, \hat{u}_{-1}}_{\text{adding $3M/2$ to index}}, \hat{u}_{0},  \hat{u}_{1}, \cdots, \hat{u}_{\frac{M}{2}}, 0, \cdots, \underbrace{0}_{(\frac{3M}{4})^{th}} \}
$$

$$
\hat{u}_{pad} = \{\hat{u}_{0},  \hat{u}_{1}, \cdots, \hat{u}_{\frac{M}{2}}, 0, \cdots, \underbrace{0}_{(\frac{3M}{4})^{th}} , 0, \cdots, 0, \hat{u}_{-\frac{M}{2}+1}, \cdots, \hat{u}_{-1}\}
$$

$$
\hat{u}_{pad} = \{\hat{u}_{pad,0},  \hat{u}_{pad, 1}, \cdots, \hat{u}_{pad, \frac{M}{2}}, 0, \cdots, 0, \hat{u}_{pad,M+1}, \cdots, \hat{u}_{pad,\frac{3M}{2}-1}\}
$$

This padded vector will be transformed back to physical space where it will be multiplied by another physical variable.

$$
u_{pad} = \{u_{pad,0},  u_{pad, 1}, \cdots, u_{pad, \frac{M}{2}}, \cdots, u_{pad,M+1}, \cdots, u_{pad,\frac{3M}{2}-1}\}
$$

![title](./1D_aliasing/Slide1.png)
![title](./2D_aliasing/Slide1.png)


# Pad and unpad

In [ ]:
def pad(f):
        ly, lx = f.shape
        lxp = (3 * lx)//2
        lyp = (3 * ly)//2
        fp = np.zeros([lyp, lxp], dtype=complex)
        
        fp[0:ly//2+1, 0:lx//2+1] = f[0:ly//2+1, 0:lx//2+1]
        fp[0:ly//2+1, lx::     ] = f[0:ly//2+1, lx//2::  ]
        fp[ly::     , 0:lx//2+1] = f[ly//2::  , 0:lx//2+1]
        fp[ly::     , lx::     ] = f[ly//2::  , lx//2::  ]
        return fp

def unpad(fp):
        lyp, lxp = fp.shape
        lx = (2 * lxp)//3
        ly = (2 * lyp)//3
        f = np.zeros([ly, lx], dtype=complex)

        f[0:ly//2+1, 0:lx//2+1] = fp[0:ly//2+1, 0:lx//2+1]
        f[0:ly//2+1, lx//2::  ] = fp[0:ly//2+1, lx::     ]
        f[ly//2::  , 0:lx//2+1] = fp[ly::     , 0:lx//2+1]
        f[ly//2::  , lx//2::  ] = fp[ly::     , lx::     ]

        return f

In [ ]:
x = np.arange(200).reshape(10,20)

fig = plt.figure(figsize=(9,3))

axs = fig.add_subplot(1,3,1)
axs.pcolor(x)
axs.set_title("x")

axs = fig.add_subplot(1,3,2)
axs.pcolor(np.real(pad(x)))
axs.set_title("x -> xp with padding")

axs = fig.add_subplot(1,3,3)
img = axs.pcolor(np.real(unpad(pad(x))))
axs.set_title("xp -> x without padding")

plt.colorbar(img)

print("x.shape:", x.shape, ", xp.shape", pad(x).shape, ", unpad(xp).shape:", unpad(pad(x)).shape)

In [ ]:
class FlowField:
    def __init__(self, grid, nu = 5e-4, pad = True, dt = None):
        self.nu =  nu
        self.grid  = grid
        self.t     = 0        
        self.u     = 0 
        self.v     = 0
        self.omega = 0
        self.psi   = 0
        self.omegahat = 0
        self.pad = pad
        if dt:
            self.dt = dt
        else:
            self.dt = 1/(self.grid.nx*4)

        self.set_initial_condition()
        
        # Set wavenumbers
        self.kx = self.set_wavenumbers(self.grid.nx, self.grid.lx)
        self.ky = self.set_wavenumbers(self.grid.ny, self.grid.ly)
        self.kxp = self.set_wavenumbers(self.grid.nxp, self.grid.lxp)
        self.kyp = self.set_wavenumbers(self.grid.nyp, self.grid.lyp)

        # Create wavenumber grid 
        self.kxg , self.kyg = np.meshgrid(self.kx,self.ky)     # no padding
        self.kxgp , self.kygp = np.meshgrid(self.kxp,self.kyp) # with padding

    def set_wavenumbers(self, n, L):
        k = np.hstack([np.arange(n/2+1), np.arange(-n/2+1,0)])
        k = 2*np.pi*k/L 
        return k


    def set_initial_condition(self, random : bool = True, nv = 100, case = 1):
        # If random == True 
        #   nv number of random Taylor vortics 
        #   will be used as initial condition.
        # If random == False
        #   case = 0 : Center vortex
        #   case = 1 : Two vortices equally spaced from 
        #              center along x-axis  
        print("Setting initial condition.")
        self.omegahat = 0

        if random:
            for _ in range(nv):
                x0 = self.grid.lx * np.random.rand()
                y0 = self.grid.ly * np.random.rand()
                a0 = self.grid.lx/20
                Umax = 2*np.random.rand() - 1

                self.omegahat += TaylorVortex(self.grid, x0=x0, y0=y0, a0=a0, Umax=Umax)
        else:
            if case == 1:
                self.omegahat = TaylorVortex(grid = self.grid,
                                           x0   = self.grid.lx/2,
                                           y0   = self.grid.ly/2,
                                           a0   = self.grid.ly/8,
                                           Umax = 1)
            elif case ==2:
                self.omegahat = TaylorVortex(grid  = self.grid, 
                                             x0    = self.grid.lx/2,
                                             y0    = 0.4*self.grid.ly,
                                             a0    = self.grid.lx/10,
                                             Umax = 1)
                self.omegahat+= TaylorVortex(grid  = self.grid, 
                                             x0    = self.grid.lx/2,
                                             y0    = 0.6*self.grid.ly,
                                             a0    = self.grid.lx/10,
                                             Umax = 1)    
            else:
                raise Exception("Invalid case.")
        print("Setting initial condition successful.")
        
    def fourier2physical(self):
        with np.errstate(divide='ignore', invalid='ignore'):
            psihat = self.omegahat/(self.kxg**2 + self.kyg**2) 
        psihat[0,0] = 0

        uhat = psihat * 1j * self.ky.reshape(-1,1) 
        vhat = psihat * 1j * self.kx.reshape(1,-1) 

        self.psi = np.fft.ifft2(psihat)
        self.u = np.fft.ifft2(uhat)
        self.v = np.fft.ifft2(vhat)
        self.omega = np.fft.ifft2(self.omegahat)

    def rhs(self, t, omegahat):
        # Note: here we have used omegahat as input argument
        # which will be not tied to object field of omegahat 
        # and can be updated external integrator like rk4 etc.
        # and this function should be sufficient to compute rhs 
        # based on omegahat.
        return -self.nu * (self.kxg**2 + self.kyg**2) * omegahat + self.advection(omegahat)
    
    def advection(self, omegahat):
        # unpadded
        with np.errstate(divide='ignore', invalid='ignore'):
            psihat = omegahat/(self.kxg**2 + self.kyg**2) 
        psihat[0,0] = 0

        d_omega_hat_dx = omegahat * 1j * self.kxg 
        d_omega_hat_dy = omegahat * 1j * self.kyg

        uhat = psihat * 1j * self.kyg
        vhat =-psihat * 1j * self.kxg

        if self.pad:
            up = np.fft.ifft2(pad(uhat))
            vp = np.fft.ifft2(pad(vhat))

            d_omega_dxp = np.fft.ifft2(pad(d_omega_hat_dx))
            d_omega_dyp = np.fft.ifft2(pad(d_omega_hat_dy))

            adv = unpad(np.fft.fft2( -up* d_omega_dxp - vp* d_omega_dyp ))*1.5*1.5
            # Multiplication by 1.5
            # FFT gives 1/M \sum ... for time series of length M.
            # With padding it outputs 2/(3M) \sum ... 
            # while unpadding we expect 2/(3M) \sum ... -> 1/M \sum ...
            # Thus need to multiply by 3/2 as, (3/2) x [2/(3M) \sum ...] -> 1/M \sum ...
        else:
            u = np.fft.ifft2(uhat)
            v = np.fft.ifft2(vhat)

            d_omega_dx = np.fft.ifft2(d_omega_hat_dx)
            d_omega_dy = np.fft.ifft2(d_omega_hat_dy)

            adv = unpad(np.fft.fft2( -up* d_omega_dx - vp* d_omega_dy ))
        return adv

    def march(self, dt=None):
        if dt == None:
            dt = self.dt

        self.omegahat = rk4(self.rhs, self.omegahat, self.t, self.dt)
        self.fourier2physical()

In [ ]:
grid = CartesianGrid(lx=1, ly=1, nx=128, ny=128)
flow = FlowField(grid=grid)
flow.set_initial_condition(random=False, case=2)

fig = plt.figure()
axs = fig.add_subplot(1,1,1)
axs.pcolor(flow.grid.X,flow.grid.Y,np.real(flow.omegahat))

In [ ]:
flow.fourier2physical()
fig = plt.figure()
axs = fig.add_subplot(1,1,1)
axs.pcolor(flow.grid.X,flow.grid.Y,np.real(flow.omega))

In [ ]:
for i in range(1000):
    flow.march()

In [ ]:
flow.fourier2physical()
fig = plt.figure()
axs = fig.add_subplot(1,1,1)
axs.pcolor(flow.grid.X,flow.grid.Y,np.real(flow.omega))

In [ ]:
flow.fourier2physical()
fig = plt.figure()
axs = fig.add_subplot(1,1,1)
axs.pcolor(flow.grid.X,flow.grid.Y,np.real(flow.u))

In [ ]:
flow.fourier2physical()
fig = plt.figure()
axs = fig.add_subplot(1,1,1)
axs.pcolor(flow.grid.X,flow.grid.Y,np.real(flow.v))

In [ ]:
flow.fourier2physical()
fig = plt.figure()
axs = fig.add_subplot(1,1,1)
axs.pcolor(flow.grid.X,flow.grid.Y,np.real(flow.psi))